### import

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import matplotlib.pyplot as plt

In [3]:
data_path = 'datas/'
train_path = os.path.join(data_path, 'train')
val_path = os.path.join(data_path, 'validation')

train_cats_path = os.path.join(train_path, 'cats')
train_dogs_path = os.path.join(train_path, 'dogs')
val_cats_path = os.path.join(val_path, 'cats')
val_dogs_path = os.path.join(val_path, 'dogs')

In [12]:
num_train_cats = len(os.listdir(train_cats_path))
num_train_dogs = len(os.listdir(train_dogs_path))

num_val_cats = len(os.listdir(val_cats_path))
num_val_dogs = len(os.listdir(val_dogs_path))

num_train = num_train_cats + num_train_dogs
num_val = num_val_cats + num_val_dogs

print('train cats:', num_train_cats)
print('train dogs:', num_train_dogs)
print('validation cats:', num_val_cats)
print('validation dogs:', num_val_dogs)
print('train:', num_train)
print('validation:', num_val)

train cats: 1000
train dogs: 1000
validation cats: 500
validation dogs: 500
train: 2000
validation: 1000


In [5]:
batch_size = 128
epochs = 15
IMG_HEIGHT = 150
IMG_WIDTH = 150

In [14]:
train_image_generator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=.15,
    height_shift_range=.15,
    horizontal_flip=True,
    zoom_range=0.5)
val_image_generator = ImageDataGenerator(rescale=1./255)

train_data_gen = train_image_generator.flow_from_directory(
    batch_size=batch_size,             
    directory=train_path, 
    shuffle=True,   
    target_size=(IMG_HEIGHT, IMG_WIDTH), 
    class_mode='binary')

val_data_gen = val_image_generator.flow_from_directory(
    batch_size=batch_size,             
    directory=val_path, 
    target_size=(IMG_HEIGHT, IMG_WIDTH), 
    class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [21]:
checkpoint_path = 'training1/cp-{epoch:04d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5
)

In [18]:
def create_model():
    model = models.Sequential([
        layers.Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
        layers.MaxPooling2D(),
        layers.Dropout(0.2),
        layers.Conv2D(32, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Dropout(0.2),
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy'])
    return model

In [19]:
model = create_model()
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 150, 150, 16)      448       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 75, 75, 16)        0         
_________________________________________________________________
dropout (Dropout)            (None, 75, 75, 16)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 75, 75, 32)        4640      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 37, 37, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 37, 37, 64)        18496     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 18, 18, 64)       

In [17]:
model_log = model.fit_generator(
    train_data_gen,
    steps_per_epoch=num_train,
    epochs=epochs,
    callbacks=[cp_callback],
    validation_data=val_data_gen,
    validation_steps=num_val
)

Epoch 1/15
   6/2000 [..............................] - ETA: 1:01:58 - loss: 1.3713 - accuracy: 0.5091

KeyboardInterrupt: 

In [ ]:
model.save('model.h5')